Loading data, importing libraries, and importing utils

In [ ]:
from typing import *
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from models import SimpleModel
import torchvision.transforms.v2  as F
from torchviz import make_dot
import tqdm

from sklearn.svm import LinearSVC
from main import train,test,extract_features,train_with_patches,test_patches,extract_patch_features
from fisher_vector import neural_based_fisher
from IPython.display import clear_output
from torch.utils.data import TensorDataset, DataLoader

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"

os.environ["CUDA_VISIBLE_DEVICES"]="0"

os.environ['JOBLIB_TEMP_FOLDER'] = '/home/msiau/workspace/jventosa/PostTFG/Master/project-1/Week2/joblib'


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)


#### MLP on small images

Entire Training

In [ ]:
def train_simple_model(model,model_name,train_loader,test_loader):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    num_epochs = 100

    train_losses, train_accuracies = [], []
    test_losses, test_accuracies = [], []

    best_test_loss = 10_000_000
    train_loss_at_best = 10_000_000
    best_test_accuracy =10_000_000
    best_epoch = 0
    for epoch in tqdm.tqdm(range(num_epochs), desc="TRAINING THE MODEL"):
        train_loss, train_accuracy = train(model, train_loader, criterion, optimizer, device)
        test_loss, test_accuracy = test(model, test_loader, criterion, device)

        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)
        test_losses.append(test_loss)
        test_accuracies.append(test_accuracy)

        if test_loss < best_test_loss:
            best_test_loss = test_loss
            train_loss_at_best = train_accuracy
            best_test_accuracy = test_accuracy
            best_epoch = epoch+1
            torch.save(model.state_dict(), f"SimpleModel/{model_name}.pth")

        print(f"Epoch {epoch + 1}/{num_epochs} - "
                f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, "
                f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")
    return best_test_accuracy, train_loss_at_best, best_epoch

In [ ]:
optimal_image_size = 16
transformation  = F.Compose([
                                F.ToImage(),
                                F.ToDtype(torch.float32, scale=True),
                                F.Resize(size=(optimal_image_size, optimal_image_size)),
                            ])

data_train = ImageFolder("../places_reduced/train", transform=transformation)
data_test = ImageFolder("../places_reduced/val", transform=transformation) 

train_loader = DataLoader(data_train, batch_size=256, pin_memory=True, shuffle=True, num_workers=8)
test_loader = DataLoader(data_test, batch_size=128, pin_memory=True, shuffle=False, num_workers=8)

#### SVM on layers from model

We will use the same model as last cell, and try some taking features from each layer

In [ ]:
model = SimpleModel(input_d=C*16*16,hidden_layers_n=1, hidden_d=256, output_d=11)

model = model.to(device)
model.load_state_dict(torch.load(f"SimpleModel/768_input_1_layers_256_dimension.pth", weights_only=True))

train_feats, train_labels = extract_features(model, train_loader, device, layer_id)
test_feats, test_labels = extract_features(model, test_loader, device, layer_id)

svm = LinearSVC(C=1.0)
svm.fit(train_feats, train_labels)
svm_acc = svm.score(test_feats, test_labels)

print(f"SVM accuracy using layer {layer_id}: {svm_acc:.4f}")

#### Patch predictions joined

In [ ]:
C, H, W = np.array(data_train[0][0]).shape

patch_size = int(H / 4)

hidden_layers_n = 2

model = SimpleModel(input_d=C*patch_size*patch_size,hidden_layers_n=hidden_layers_n, hidden_d=300, output_d=11)


model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 20

train_losses, train_accuracies = [], []
test_losses, test_accuracies = [], []

best_test_loss = 10_000_000

for epoch in tqdm.tqdm(range(num_epochs), desc="TRAINING THE PATCH BASED MODEL"):
    train_loss, train_accuracy = train_with_patches(model, train_loader, criterion, optimizer, device,patch_size)
    test_loss, test_accuracy = test_patches(model, test_loader, criterion, device,patch_size)

    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)

    if test_loss < best_test_loss:
        best_test_loss = test_loss
        os.makedirs(f"PatchModel_{patch_size}",exist_ok=True)
        torch.save(model.state_dict(), f"PatchModel_{patch_size}/BestWeights.pth")

    print(f"Epoch {epoch + 1}/{num_epochs} - "
            f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, "
            f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

In [ ]:
C, H, W = np.array(data_train[0][0]).shape

patch_size = int(H / 4)

hidden_layers_n = 2

model = SimpleModel(input_d=C*patch_size*patch_size,hidden_layers_n=hidden_layers_n, hidden_d=300, output_d=11)

model = model.to(device)
model.load_state_dict(torch.load(f"PatchModel_{patch_size}/BestWeights.pth", weights_only=True))

criterion = nn.CrossEntropyLoss()

test_loss, test_accuracy = test_patches(model, test_loader, criterion, device,patch_size)

print(test_loss,test_accuracy)

#### Passing patch features to BOVW

Our best method was ___ 

In [ ]:


C, H, W = np.array(data_train[0][0]).shape

patch_size = int(H / 4)

hidden_layers_n = 2

model = SimpleModel(input_d=C*patch_size*patch_size,hidden_layers_n=hidden_layers_n, hidden_d=300, output_d=11)

model = model.to(device)
model.load_state_dict(torch.load(f"PatchModel_{patch_size}/BestWeights.pth", weights_only=True))


for layer_id in range(hidden_layers_n+1):
    train_feats, train_labels = extract_patch_features(model, train_loader, device, layer_id,patch_size)
    test_feats, test_labels = extract_patch_features(model, test_loader, device, layer_id,patch_size)
    print(train_feats.shape)
    print(train_labels.shape)
    acc , _ = neural_based_fisher(train_feats, test_feats, train_labels, test_labels)

    print(f"BOVW accuracy using layer {layer_id}: {acc:.4f}")